In [1]:
pip install apimoex

In [2]:
import requests
import apimoex
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px


with requests.Session() as session:
    data = apimoex.get_board_history(session, 'GAZP')
    df = pd.DataFrame(data)
    df.set_index('TRADEDATE', inplace = True)
    print(df.head(), '\n')
    print(df.tail(),'\n')
    df.info

           BOARDID   CLOSE    VOLUME         VALUE
TRADEDATE                                         
2014-06-09    TQBR  144.40  40347300  5.851239e+09
2014-06-10    TQBR  144.75  41460990  5.956067e+09
2014-06-11    TQBR  146.40  39418520  5.725373e+09
2014-06-16    TQBR  145.20  77165360  1.113453e+10
2014-06-17    TQBR  144.77  44370530  6.436430e+09 

           BOARDID   CLOSE     VOLUME         VALUE
TRADEDATE                                          
2024-06-20    TQBR  115.92  176304340  1.984459e+10
2024-06-21    TQBR  115.40   45955860  5.319741e+09
2024-06-24    TQBR  113.67   27459870  3.146136e+09
2024-06-25    TQBR  114.97   32091500  3.648872e+09
2024-06-26    TQBR  117.00   39678830  4.609402e+09 



In [4]:
df_n = df[['CLOSE']]
df_n

,CLOSE
TRADEDATE,
2014-06-09,144.40
2014-06-10,144.75
2014-06-11,146.40
2014-06-16,145.20
2014-06-17,144.77
...,...
2024-06-20,115.92
2024-06-21,115.40
2024-06-24,113.67


In [5]:
fig = px.line(df_n,
              #  title = 'Изменение',
                width=1200, height=600,
                      labels={
                     "value": "Стоимость 1 акции, руб.",
                     "TRADEDATE": "Дата проведения торгов",
                     "species": "Species of Iris"
                 }

              )
fig.update_layout(showlegend=False)
fig.show()

Из графика можно видеть отсутствующие значения, когда торги не проводились (в феврале 2022 г.)

In [6]:
def buildLaggedFeatures(df_n,lag=5,dropna=True):
 if type(df_n) is pd.DataFrame:
    new_dict={}
    for col_name in df_n:
        new_dict[col_name]=df_n[col_name]
        for l in range(1,lag+1):
            new_dict['%s_lag%d' %(col_name,l)]=df_n[col_name].shift(l)
    res=pd.DataFrame(new_dict,index=df_n.index)
 else:
    return None
 if dropna:
    return res.dropna()
 else:
    return res

In [7]:
res=buildLaggedFeatures(df_n,lag=10,dropna=True)
res

,CLOSE,CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
TRADEDATE,,,,,,,,,,,
2014-06-25,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20,146.40,144.75,144.40
2014-06-26,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20,146.40,144.75
2014-06-27,149.39,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20,146.40
2014-06-30,148.96,149.39,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20
2014-07-01,148.97,148.96,149.39,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-20,115.92,113.85,118.02,121.54,121.65,119.63,117.53,119.00,122.78,123.60,124.02
2024-06-21,115.40,115.92,113.85,118.02,121.54,121.65,119.63,117.53,119.00,122.78,123.60
2024-06-24,113.67,115.40,115.92,113.85,118.02,121.54,121.65,119.63,117.53,119.00,122.78


In [8]:
train=res.loc[:'2021-06-09']
test=res.loc['2021-06-10':]

In [9]:
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.3-py2.py3-none-any.whl size=265365897 sha256=8d6adf5cf0f980d98acf7109e91ccee3e1580ec85acfcb8741d140ef14e9a668
  Stored in directory: /root/.cache/pip/wheels/c4/63/41/baa115b5255e1db3e2383bce4e2e6181746aac0b42264c242f
Successfully built h2o


In [10]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.23" 2024-04-16; OpenJDK Runtime Environment (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1); OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpb714sb5a
  JVM stdout: /tmp/tmpb714sb5a/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpb714sb5a/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.3
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_unknownUser_y3m0um
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [11]:
hdff = h2o.H2OFrame(train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
hdfp = h2o.H2OFrame(test)
hdfp.tail()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


CLOSE,CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
119.63,117.53,119,122.78,123.6,124.02,125.1,124.14,126.46,124.93,128.89
121.65,119.63,117.53,119,122.78,123.6,124.02,125.1,124.14,126.46,124.93
121.54,121.65,119.63,117.53,119,122.78,123.6,124.02,125.1,124.14,126.46
118.02,121.54,121.65,119.63,117.53,119,122.78,123.6,124.02,125.1,124.14
113.85,118.02,121.54,121.65,119.63,117.53,119,122.78,123.6,124.02,125.1
115.92,113.85,118.02,121.54,121.65,119.63,117.53,119,122.78,123.6,124.02
115.4,115.92,113.85,118.02,121.54,121.65,119.63,117.53,119,122.78,123.6
113.67,115.4,115.92,113.85,118.02,121.54,121.65,119.63,117.53,119,122.78
114.97,113.67,115.4,115.92,113.85,118.02,121.54,121.65,119.63,117.53,119
117,114.97,113.67,115.4,115.92,113.85,118.02,121.54,121.65,119.63,117.53


In [13]:
y = "CLOSE"
x_train = hdff.columns
x_train.remove(y)
aml = H2OAutoML(max_runtime_secs=30, nfolds = 5)
aml.train(x = x_train, y = y, training_frame = hdff)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),2/3
# GBM base models (used / total),0/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [14]:
pred = aml.predict(hdfp)
pred.tail()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict
118.229
119.198
120.904
121.412
119.135
115.513
115.826
115.472
114.159
114.806


In [15]:
aml.leaderboard

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_BestOfFamily_1_AutoML_1_20240627_183924,2.94799,8.69066,2.01377,0.0168435,8.69066
StackedEnsemble_BestOfFamily_2_AutoML_1_20240627_183924,2.94866,8.6946,2.01543,0.0168418,8.6946
StackedEnsemble_AllModels_1_AutoML_1_20240627_183924,2.94917,8.69758,2.01566,0.0168468,8.69758
GLM_1_AutoML_1_20240627_183924,2.96027,8.76321,2.02007,0.0169113,8.76321
XGBoost_1_AutoML_1_20240627_183924,3.42127,11.7051,2.44751,0.0194871,11.7051
XGBoost_2_AutoML_1_20240627_183924,3.48275,12.1295,2.42207,0.0197271,12.1295
DRF_1_AutoML_1_20240627_183924,3.58425,12.8468,2.51038,0.0203585,12.8468
GBM_1_AutoML_1_20240627_183924,4.28013,18.3195,2.74451,0.0222114,18.3195
XRT_1_AutoML_1_20240627_183924,4.98373,24.8375,3.32108,0.0281327,24.8375
GBM_4_AutoML_1_20240627_183924,7.29659,53.2403,5.49313,0.0397835,53.2403


In [ ]:
#aml.explain(hdfp)

In [16]:
predict = pred.as_data_frame() # перегоняем обратно в датафрейм

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1981: H2ODependencyWarning:

Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)




In [17]:
r1 = pd.DataFrame(columns=['Дата', 'Значение', 'Тип'])
r1['Дата']=train.index.values
r1['Значение']=train[['CLOSE']].values
r1['Тип'] = 'Обучающая выборка'

In [18]:
r2 = pd.DataFrame(columns=['Дата', 'Значение', 'Тип'])
r2['Дата']=test.index.values
r2['Значение']=test[['CLOSE']].values
r2['Тип'] = 'Тестовая выборка'

In [19]:
r3 = pd.DataFrame(columns=['Дата', 'Значение', 'Тип'])
r3['Дата']=test.index.values
r3['Значение']=predict.values
r3['Тип'] = 'Предсказание модели'

In [20]:
r= pd.concat([r1,r2,r3])


In [21]:
fig = px.line(r, x="Дата", y="Значение", color='Тип',
                width=1200, height=600,
                         labels={
                     "Значение": "Стоимость 1 акции, руб.",
                     "Дата": "Дата проведения торгов",

                 }
              )
fig.update_layout(showlegend=True)
fig.show()

In [22]:
col = res.columns [1:]
day = res.iloc[:,:-1].tail(1).values
day = pd.DataFrame(day, columns=col)
day

,CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
0,117.0,114.97,113.67,115.4,115.92,113.85,118.02,121.54,121.65,119.63


In [23]:
day = h2o.H2OFrame(day)
day


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
117,114.97,113.67,115.4,115.92,113.85,118.02,121.54,121.65,119.63


In [24]:
# прозноз на 1 день (на завтра)
pred1 = aml.predict(day)
pred1.tail()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict
116.429
